# Create migration matrix
VGsim needs a matrix with entries containing probability of an individual moving between locations per unit time, this notebook
obtains that by calculating number of people flying between countries through reversing an effective distance matrix that is available in some supplementary materials https://www.science.org/doi/10.1126/science.1245200#supplementary-materials. 

### Load libraries and datasets

In [342]:
import pandas as pd
import numpy as np

# Alpha2 code of where to start the pandemic, look in the effective distance matrix for the available countries.
pandemic_start_alpha2 = 'CN'
# Uniform probability of staying in any country per unit time.
in_country_probability = 0.95

# Effective distance data

effective_distances = pd.read_csv('original_data/effective.distance.matrix.country.csv', header=0)
effective_distances.drop(effective_distances.columns[0], axis=1, inplace=True)

# Population sizes data

population_sizes = pd.read_csv('output/census_2013.csv', header=0)
alpha2_codes = effective_distances.columns

# Sampling data (obtained from sampling_multipliers.ipynb)

sampling_multipliers = pd.read_csv('output/sampling_multipliers.csv', header=0, index_col=0)

effective_distances.head()

,AE,AF,AG,AI,AL,AM,AO,AR,AS,AT,...,VI,VN,VU,WF,WS,YE,YT,ZA,ZM,ZW
0,0.000000,5.614035,11.531617,15.902166,10.178471,7.769530,7.319591,9.130665,13.846666,6.344803,...,10.779283,7.031367,10.655119,14.960192,11.888486,6.435626,12.316025,5.262377,7.236649,8.955652
1,1.548074,0.000000,13.079690,17.450240,10.173316,9.317603,8.867664,10.678739,15.394739,5.369434,...,12.327357,8.579441,12.203193,16.508266,13.436560,7.983700,13.864099,6.810451,8.784722,10.503726
2,7.181742,12.795778,0.000000,5.366969,11.139288,13.218427,12.275860,8.597538,11.985323,8.737326,...,7.430663,11.554362,14.098395,16.611105,13.239721,13.617368,14.920257,8.765838,12.203931,12.459113
3,8.968363,14.582398,2.224032,0.000000,13.363319,14.356298,14.254211,9.569931,12.957716,10.961358,...,2.213855,12.526755,15.070788,17.583498,14.212114,15.403988,14.410679,10.989870,14.427962,14.683145
4,6.413046,10.472095,11.760670,17.127640,0.000000,10.349453,11.707611,8.216086,15.163964,3.566389,...,12.096582,11.296419,17.068165,18.930315,16.418362,11.336016,12.905280,10.016969,13.455061,13.435707


## Flux matrix
Invert effective distance function to get flux. Off diagonals matter only, since diagonal is zero in effective distance matrix.

In [343]:
flux_matrix = effective_distances.apply(lambda x: np.exp(1 - x))

## Clean flux matrix
Remove rows and columns of countries in flux matrix that aren't in population dataset.

In [344]:
missing_countries_columns = []
missing_countries_rows = []
for index, alpha2 in enumerate(alpha2_codes):
    country_mask = population_sizes['alpha2'] == alpha2
    country_data = population_sizes[country_mask]
    if len(country_data['population']) == 0:
        missing_countries_columns.append(alpha2)
        missing_countries_rows.append(index)
flux_matrix = flux_matrix.drop(missing_countries_columns, axis=1).drop(flux_matrix.index[missing_countries_rows])
flux_matrix.index = flux_matrix.columns

# Arbritralily say that 85% of people in each country stay in that country each unit of time. If migration rates are unrealistically high, this number is too low.
np.fill_diagonal(flux_matrix.values, in_country_probability)

In [345]:
row_normalised = flux_matrix.div(flux_matrix.sum(axis=0), axis=1)

# Place starting place at (0,0)
By default, the first entry in the population data is where the pandemic starts (in VGsim), so we need to swap the first row and first column with the row and column of the country we want to start in. 


In [346]:
# Get column index (and by symmetry also row index) of starting country 
target_start_index = row_normalised.columns.get_loc(pandemic_start_alpha2)
original_start_name = row_normalised.columns[0]
# Swap the columns of the start country and the first country 
col_list = list(row_normalised)
col_list[0], col_list[target_start_index] = col_list[target_start_index], col_list[0]
row_normalised.columns = col_list

# Swap the rows 
ordered_proportions = row_normalised.reindex(col_list)

In [347]:
filtered_pop = population_sizes[population_sizes['alpha2'].isin(alpha2_codes)]
filtered_pop.set_index('alpha2', inplace=True)
filtered_pop = filtered_pop.reindex(col_list)
filtered_pop.head()

,population,fullname
alpha2,,
CN,1371860000,China
AF,32716210,Afghanistan
AG,89236,Antigua and Barbuda
AL,2889104,Albania
AM,2889930,Armenia


## Write matrix in .mg format


In [348]:
base_file = 'output/manypop'

In [349]:
with open(base_file + '.mg', 'w') as f:
    f.write('#Migration_format_version 0.0.1\n')

(ordered_proportions.transpose()).to_csv(base_file + '.mg', header=False, index=None, mode="a", sep=' ', float_format='%.15f')

## Write population data in .pp format 

In [350]:
sampling_multipliers_complete = np.full(shape=len(filtered_pop['population']),fill_value=0.00,dtype=np.double)
outer_index = 0
for index, row in filtered_pop.iterrows():
    if index in sampling_multipliers.index:
        sampling_multipliers_complete[outer_index] = sampling_multipliers.loc[index,'proportion'].astype(float)
    outer_index+=1

In [351]:

population_data = pd.DataFrame({'size': filtered_pop['population'],
                                'contactDensity': np.full(shape=len(filtered_pop['population']), fill_value=1.00,
                                                          dtype=np.double),
                                'conDenAfterLD': np.full(shape=len(filtered_pop['population']), fill_value=0.1,
                                                         dtype=np.double),
                                'startLD': np.full(shape=len(filtered_pop['population']), fill_value=0.01,
                                                   dtype=np.double),
                                'endLD': np.full(shape=len(filtered_pop['population']), fill_value=0.002,
                                                 dtype=np.double),
                                'samplingMultiplier': sampling_multipliers_complete,
                                'fullname':filtered_pop['fullname']})
# VGsim seems to require 3 columns to be comma-seperated, while the rest are space seperated, so must merge some columns by commas: 
population_data = population_data.reindex(col_list)
population_data['id'] = range(len(filtered_pop['population']))
# Extract ID to Full country name to a separate table for relabelling countries after simulations
population_translation_data = population_data[['id','fullname']]
population_translation_data = population_translation_data.rename(columns={'id':'location'})
population_data['conDenAfterLD startLD endLD'] = population_data['conDenAfterLD'].astype(str) + "," + population_data[
    'startLD'].astype(str) + "," + population_data['endLD'].astype(str)
population_data = population_data[['id', 'size', 'contactDensity', 'conDenAfterLD startLD endLD', 'samplingMultiplier']]
population_data.head()

,id,size,contactDensity,conDenAfterLD startLD endLD,samplingMultiplier
alpha2,,,,,
CN,0,1371860000,1.0,"0.1,0.01,0.002",0.000004
AF,1,32716210,1.0,"0.1,0.01,0.002",0.000000
AG,2,89236,1.0,"0.1,0.01,0.002",0.000000
AL,3,2889104,1.0,"0.1,0.01,0.002",0.000000
AM,4,2889930,1.0,"0.1,0.01,0.002",0.000000


In [352]:
with open(base_file + '.pp', 'w') as f:
    f.write('#Population_format_version 0.0.1\n')

population_data.to_csv(base_file + '.pp', header=True, index=None, mode="a", sep=' ')
population_translation_data.to_csv(r'./output/manypop_country_ids.csv', index=None, sep=',', mode='w')

# Output
You can access the resulting three data files relative in:
 1. `/migration_data/output/manypop.mg`
2. `/migration_data/output/manypop.pp`
3. `/migration_data/output/manypop_country_codes.csv` 